In [ ]:
%pip show webdriver-manager
%pip show selenium

In [2]:
import csv
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [7]:
driver = webdriver.Chrome()
driver.get("https://cases.shc.gov.pk/")
driver.implicitly_wait(10)

# Find all the district divs urls FIRST and store them
path = '//div[@class="row mt-4 mb-5"]/div[@class="col-md-2 mb-3"]/a'
items = driver.find_elements(By.XPATH, path)

print(f"Total items found: {len(items)}")

# Store URLs and names FIRST before any navigation
district_data = []
for item in items:
    url = item.get_attribute('href')
    # Try to get the district name if needed
    try:
        name = item.find_element(By.XPATH, './/div[@class="card-header"]').text
    except:
        name = "Unknown"
    district_data.append({'url': url, 'name': name})

print("District URLs found:")
for data in district_data:
    print(f"{data['name']}: {data['url']}")

# Now process each district
for district in district_data:
    print(f"\n=== Processing {district['name']} ===")
    
    # Navigate to district page
    driver.get(district['url'])
    
    try:
        # Wait for the dropdown to load on the new page
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
        )
        
        # Find all options on THIS page
        options = driver.find_elements(By.XPATH, '//select[@id="casessearch-circuitcode"]/option')
        options = options[1:]  # Remove first placeholder option
        
        print(f"Number of court options found: {len(options)}")
        
        # Get values
        values = [option.get_attribute('value') for option in options]
        print("Court values are:", values)
        
        # Process each court value
        for value in values:
            try:
                print(f"Processing court value: {value}")
                
                # Set the value using JavaScript
                driver.execute_script(f"document.getElementById('casessearch-circuitcode').value = '{value}';")
                
                # Click search button
                search_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, "submit_search"))
                )
                search_button.click()
                
                # Wait for results page
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.XPATH, "//table | //tbody | //div[contains(@class, 'table')]"))
                )
                print(f"Results page loaded for value {value}")
                
                # Do your data extraction here
                print(f"Current results URL: {driver.current_url}")
                
                # Go back to the district search page
                driver.back()
                
                # Wait for the search page to reload
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
                )
                print("Returned to district search page")
                
            except Exception as e:
                print(f"Error processing court value {value}: {e}")
                # Try to recover by going back to district page
                try:
                    driver.get(district['url'])
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
                    )
                except:
                    print("Could not recover, moving to next district")
                    break
                
    except Exception as e:
        print(f"Error loading district page {district['url']}: {e}")
        continue

driver.quit()

Total items found: 6
District URLs found:
SHC, Principal Seat Karachi: https://cases.shc.gov.pk/khi
SHC, Bench at Sukkur: https://cases.shc.gov.pk/suk
SHC, Circuit Court Hyderabad: https://cases.shc.gov.pk/hyd
SHC, Circuit Court Larkana: https://cases.shc.gov.pk/lar
SHC, Circuit Court Mirpurkhas: https://cases.shc.gov.pk/mpkhas/web/index.php?r=cases/search-result&CasesSearch[CIRCUITCODE]=13
District Courts of Sindh: https://cases.districtcourtssindh.gos.pk/

=== Processing SHC, Principal Seat Karachi ===
Number of court options found: 10
Court values are: ['1', '2', '3', '4', '5', '-1', '13', '14', '6', '7']
Processing court value: 1
Results page loaded for value 1
Current results URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch-result&CasesSearch%5BCASENO%5D=&CasesSearch%5BCASEYEAR%5D=&CasesSearch%5BCASENAMECODE%5D=&CasesSearch%5BBENCH%5D=&CasesSearch%5BCIRCUITCODE%5D=1&CasesSearch%5BMATTERCODE%5D=&CasesSearch%5BPARTY%5D=&CasesSearch%5BGOVT_AGENCY_CODE%5D=&CasesSearch%

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=60465): Read timed out. (read timeout=120)

In [ ]:
driver = webdriver.Chrome() # opens up the web page
driver.get("https://cases.shc.gov.pk/khi") # sindh high court website
driver.implicitly_wait(10) # waits for the web elements to load

# Direct XPath to find all option elements of the specific select
dropdown_path = '//select[@id="casessearch-circuitcode"]/option'
options = driver.find_elements(By.XPATH, dropdown_path)
options = options[1:]
print(f"Number of options found: {len(options)}")

# has the values to access each option
values = [option.get_attribute('value') for option in options]
# values = values[1:] # remove the first place holder value

print("Values are:", values)

for value in values:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
        )
        driver.execute_script(f"document.getElementById('casessearch-circuitcode').value = '{value}';")
    
        # Click the search button
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "submit_search"))
        )
        search_button.click()
        
        print(f"Current URL: {driver.current_url}")
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, "//table"))
        )
        print(f"Results page loaded successfully for value {value}")


        # Go back to the search page
        driver.back()
        # Wait for search page to reload completely
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "casessearch-circuitcode"))
        )
        print("Returned to search page")

    except Exception as e:
         print(f"Error setting value {value}: {e}")     






Number of options found: 10
Values are: ['1', '2', '3', '4', '5', '-1', '13', '14', '6', '7']
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search page
Current URL: https://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch
Results page loaded successfully
Returned to search p

'\nhttps://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch-result&CasesSearch%5BCASENO%5D=&CasesSearch%5BCASEYEAR%5D=&CasesSearch%5BCASENAMECODE%5D=&CasesSearch%5BBENCH%5D=&CasesSearch%5BCIRCUITCODE%5D=1&CasesSearch%5BMATTERCODE%5D=&CasesSearch%5BPARTY%5D=&CasesSearch%5BGOVT_AGENCY_CODE%5D=&CasesSearch%5BFIRNO%5D=&CasesSearch%5BFIRYEAR%5D=&CasesSearch%5BPOLICESTATIONCODE%5D=&CasesSearch%5BADVOCATECODE%5D=&CasesSearch%5BisPending%5D=&CasesSearch%5BisPending%5D=3 \nhttps://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fsearch-result&CasesSearch%5BCASENO%5D=&CasesSearch%5BCASEYEAR%5D=&CasesSearch%5BCASENAMECODE%5D=&CasesSearch%5BBENCH%5D=&CasesSearch%5BCIRCUITCODE%5D=2&CasesSearch%5BMATTERCODE%5D=&CasesSearch%5BPARTY%5D=&CasesSearch%5BGOVT_AGENCY_CODE%5D=&CasesSearch%5BFIRNO%5D=&CasesSearch%5BFIRYEAR%5D=&CasesSearch%5BPOLICESTATIONCODE%5D=&CasesSearch%5BADVOCATECODE%5D=&CasesSearch%5BisPending%5D=&CasesSearch%5BisPending%5D=3      \nhttps://cases.shc.gov.pk/khi/web/index.php?r=cases%2Fs